In [27]:
import pandas as pd
from sodapy import Socrata
import datetime
import boto3
import s3fs
import json
import io
from botocore.errorfactory import ClientError

# call s3 bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket('ia-final-project-bucket')
key = 'KEY_PATH'


client = Socrata('data.cityofnewyork.us',
             'Tarfn2kl2IdBRX1RkC5rhcoOh',
             timeout = 150
            )
Previous_twoday = str(datetime.datetime.today() - datetime.timedelta(days=2))[:19]
Previous_twoday = Previous_twoday[:10] + 'T' + Previous_twoday[11:]
Previous_oneday = str(datetime.datetime.today() - datetime.timedelta(days=1))[:19]
Previous_oneday = Previous_oneday[:10] + 'T' + Previous_oneday[11:]
response = client.get("erm2-nwe9", where="created_date BETWEEN '"+Previous_twoday+"' AND '"+Previous_oneday+"'", limit=10000)
results_df = pd.DataFrame.from_records(response)
client.close()
print('Successfully getting create time from \'{}\' to \'{}\''.format(Previous_twoday, Previous_oneday))

s3 = boto3.client('s3')
try:
    file_name = "311_service_request/latest_service_request.csv"
    # check if the csv file exist in s3 bucket
    # get the existing file
    obj = s3.get_object(Bucket='ia-final-project-bucket', Key=file_name)
    df_current = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=0)
    current_data = df_current.to_csv(None).encode()
    # append
    bytes_to_write = results_df.to_csv(None, header=None).encode()
    appended_data = current_data + bytes_to_write
    # overwrite
    s3.put_object(Body=appended_data, Bucket='ia-final-project-bucket', Key=file_name)
    print('Successfully found existing file and appended data to {}'.format(file_name))
    
except ClientError:
    # Not found
    print('3')
    pathname = 'ia-final-project-bucket/'  # specify location of s3:/{my-bucket}/
    filenames = pathname + file_name  # name of the filepath and csv file
    byte_encoded_df = results_df.to_csv(None).encode()  # encodes file as binary
    s3 = s3fs.S3FileSystem(anon=False)
    with s3.open(filenames, 'wb') as file:
        file.write(byte_encoded_df)  # writes byte-encoded file to s3 location
    print("Successfull created and uploaded file to location:" + str(filenames))
    pass



Successfully getting create time from '2021-08-15T11:39:20' to '2021-08-16T11:39:20'
Successfully found existing file and appended data to 311_service_request/latest_service_request.csv


In [25]:
s3 = boto3.client('s3')
file_name = "311_service_request/latest_service_request.csv"
print('1')
# check if the csv file exist in s3 bucket
# get the existing file
obj = s3.get_object(Bucket='ia-final-project-bucket', Key=file_name)
df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=0)
df

1


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,landmark,facility_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_segment,vehicle_type,bridge_highway_direction,road_ramp,due_date
0,51541996,2021-08-15T11:14:00.000,2021-08-15T11:14:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running (WC3),10002.0,120 FORSYTH STREET,FORSYTH STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51535349,2021-08-15T11:14:00.000,2021-08-15T14:59:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running Full (WA4),10463.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51542223,2021-08-15T11:14:09.000,2021-08-15T11:14:09.000,DOB,Department of Buildings,General Construction/Plumbing,Building Permit - None,10303.0,62 SOUTH AVENUE,SOUTH AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51542263,2021-08-15T11:14:21.000,2021-08-15T18:45:30.000,NYPD,New York City Police Department,Noise - Vehicle,Engine Idling,10466.0,4328 BRUNER AVENUE,BRUNER AVENUE,...,BRUNER AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51536503,2021-08-15T11:14:27.000,2021-08-15T11:51:06.000,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Traffic,11226.0,1901 BEVERLEY ROAD,BEVERLEY ROAD,...,BEVERLEY ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,51540909,2021-08-16T01:02:45.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,10458,355 EAST 187 STREET,EAST 187 STREET,ELM PLACE,...,MARION AVENUE,EAST 187 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5144,51534278,2021-08-16T01:02:45.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,11226,802 ROGERS AVENUE,ROGERS AVENUE,MARTENSE STREET,...,CHURCH AVENUE,ROGERS AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5145,51540930,2021-08-16T01:02:54.000,DOHMH,Department of Health and Mental Hygiene,Food Establishment,No Permit or License,11372,85-09 ROOSEVELT AVENUE,ROOSEVELT AVENUE,85 STREET,...,86 STREET,ROOSEVELT AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5146,51539801,2021-08-16T01:03:03.000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,10458,2640 GRAND CONCOURSE,GRAND CONCOURSE,EAST 192 STREET,...,EAST 193 STREET,GRAND CONCOURSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
pd.set_option('display.max_columns', None)
df.head(5)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2,location_type,landmark,facility_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_segment,vehicle_type,bridge_highway_direction,road_ramp,due_date
0,51541996,2021-08-15T11:14:00.000,2021-08-15T11:14:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running (WC3),10002.0,120 FORSYTH STREET,FORSYTH STREET,BROOME ST,DELANCEY ST,ADDRESS,NEW YORK,Closed,The Department of Environmental Protection det...,2021-08-15T11:14:00.000,03 MANHATTAN,1.004190e+09,MANHATTAN,986354.0,201313.0,ONLINE,Unspecified,MANHATTAN,40.71923415734852,-73.99240983513386,"{'latitude': '40.71923415734852', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51535349,2021-08-15T11:14:00.000,2021-08-15T14:59:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running Full (WA4),10463.0,NaN,NaN,NaN,NaN,INTERSECTION,NEW YORK,Closed,The Department of Environmental Protection det...,2021-08-15T14:59:00.000,08 BRONX,NaN,MANHATTAN,1009403.0,258886.0,ONLINE,Unspecified,MANHATTAN,40.877220724185456,-73.90904511056735,"{'latitude': '40.877220724185456', 'longitude'...",MARBLE HILL AVENUE,WEST 228 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51542223,2021-08-15T11:14:09.000,2021-08-15T11:14:09.000,DOB,Department of Buildings,General Construction/Plumbing,Building Permit - None,10303.0,62 SOUTH AVENUE,SOUTH AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,Open,Your Service Request has been submitted to the...,2021-08-15T00:00:00.000,01 STATEN ISLAND,5.012630e+09,STATEN ISLAND,938151.0,171937.0,UNKNOWN,Unspecified,STATEN ISLAND,40.63848398181653,-74.16610066612385,"{'latitude': '40.63848398181653', 'longitude':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51542263,2021-08-15T11:14:21.000,2021-08-15T18:45:30.000,NYPD,New York City Police Department,Noise - Vehicle,Engine Idling,10466.0,4328 BRUNER AVENUE,BRUNER AVENUE,PITMAN AVENUE,NEREID AVENUE,NaN,BRONX,Closed,The Police Department responded and upon arriv...,2021-08-15T22:45:34.000,12 BRONX,2.050530e+09,BRONX,1026007.0,266047.0,ONLINE,Unspecified,BRONX,40.89681245263957,-73.84895940923131,"{'latitude': '40.89681245263957', 'longitude':...",PITMAN AVENUE,NEREID AVENUE,Street/Sidewalk,BRUNER AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51536503,2021-08-15T11:14:27.000,2021-08-15T11:51:06.000,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Traffic,11226.0,1901 BEVERLEY ROAD,BEVERLEY ROAD,EAST 19 STREET,OCEAN AVENUE,NaN,BROOKLYN,Closed,The Police Department responded to the complai...,2021-08-15T15:51:08.000,14 BROOKLYN,3.051230e+09,BROOKLYN,995063.0,174179.0,MOBILE,Unspecified,BROOKLYN,40.6447511539938,-73.96103572346502,"{'latitude': '40.644751153993795', 'longitude'...",EAST 19 STREET,OCEAN AVENUE,Street/Sidewalk,BEVERLEY ROAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
